In [1]:
%run ../../Environment/environment.ipynb

e:\Studienarbeit_GitHub\Studienarbeit\Agents\PPO
Notebook ausgeführt


# 1. Bibliotheken importieren

In [2]:
import numpy as np
import pandas as pd
import joblib
import random
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.callbacks import CheckpointCallback
from sklearn.preprocessing import StandardScaler
from stable_baselines3.common.vec_env import DummyVecEnv
import torch

# 2. Seed setzen

In [ ]:
seed = 42
SEED  = seed % (2**32 - 1)
print(f"SEED: {SEED}")

SEED: 42


# 3. Daten einlesen

In [4]:
# -------------------------------
# CSV Datem einlesen
# -------------------------------
train_data = pd.read_csv("../../Transform_data/stand_data/2023-2018_stand_data.csv")
train_data.drop('datetime', axis=1, inplace=True)

test_data = pd.read_csv("../../Transform_data/stand_data/2025-2024_stand_data.csv")
test_data.drop('datetime', axis=1, inplace=True)

if(train_data is not None and test_data is not None):
    print("Daten erfolgreich eingelesen")

Daten erfolgreich eingelesen


# 4. Parallele Umgebungen erstellen für das Training

In [ ]:
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.vec_env import VecNormalize

def make_env():
    return TradingEnv(
        data=train_data,
        initial_cash=10_000,
        window_size=336,
        scaler_path="../../Transform_data/scaler.pkl",
        default_seed=SEED
    )

n_envs = 8  # Mehr parallele Umgebungen (8, 16 oder sogar 32 testen!)
env = SubprocVecEnv([make_env for _ in range(n_envs)])

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)
env.training = True  # Sicherstellen, dass Normalisierung aktiv ist

# 5. Erstellen des Agenten

In [ ]:
from torch import nn  # Für die Netzwerkarchitektur

# Definiere das neuronale Netz
policy_kwargs = dict(
    net_arch=[dict(pi=[128, 128], vf=[128, 128])],  # Zwei Layer mit 128 Neuronen
    activation_fn=nn.ReLU,  # Verwende ReLU als Aktivierungsfunktion
)

# Erstelle den PPO-Agenten mit verbesserten Einstellungen
model = PPO(
    "MlpPolicy",
    env,
    learning_rate=0.0001,  # Lernrate
    gamma=0.99,  # Discount-Faktor
    clip_range=0.2,  # PPO-Clip-Parameter
    ent_coef=0.01,  # Entropie-Koeffizient
    n_steps=4_096,  # WICHTIG: Mehr Schritte pro Update → GPU-Auslastung steigt
    batch_size=4_096,  # WICHTIG: Große Batch-Größe → GPU rechnet effizienter
    policy_kwargs=policy_kwargs,
    verbose=1,
    seed=SEED,
    device="cuda",  # Nutzt die GPU!
    #tensorboard_log="./tensorboard_log/"  # Optional: Logging für TensorBoard
)

Using cpu device


c:\Users\tlfin\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


# 6. Modell trainieren und speichern

In [ ]:
model.learn(total_timesteps=10000)

model.save("ppo_trading_model")

-------------------------------
| time/              |        |
|    fps             | 1288   |
|    iterations      | 1      |
|    time_elapsed    | 203    |
|    total_timesteps | 262144 |
-------------------------------
